# Import data

verify our CNN model (trained from senior project) and compare with SolCast (validate on senior project test data)

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns

In [13]:
CNN_data = pd.read_csv('/Users/khunnoot/Desktop/year4/senior_project/solarmap_project/estimation_irradiance/Ihat_2layerCNN.csv',
           parse_dates = ['datetime'])
CNN_data.rename(columns = {'datetime':'Datetime', 'site':'site_name'},inplace = True )
CNN_data.Datetime = pd.to_datetime(CNN_data.Datetime)
CNN_data.set_index('Datetime', inplace = True)
CNN_data.index = CNN_data.index.tz_localize('Asia/Bangkok')
CNN_data.reset_index(inplace=True)

In [14]:
solcast30 = pd.read_csv('/Users/khunnoot/Downloads/solcast_ISL30_GHI_14.9486_102.0542_fixed_23_180_PT15M.csv',
            parse_dates = ['period_end'], usecols = lambda columns : (columns != 'period_start')& (columns != 'period')
                       &(columns != 'Unnamed: 0'))
solcast30['site_name'] = 'ISL030'
solcast48 = pd.read_csv('/Users/khunnoot/Downloads/solcast_ISL48_GHI_14.9486_102.0542_fixed_23_180_PT15M.csv',
            parse_dates = ['period_end'], usecols = lambda columns : (columns != 'period_start')& (columns != 'period'))
solcast48['site_name'] = 'ISL048'


solcast_data = pd.concat([solcast30, solcast48])
solcast_data.rename(columns = {'period_end':'Datetime','ghi':'solcast_GHI'},inplace = True)
solcast_data['Datetime'] = pd.to_datetime(solcast_data['Datetime'])
solcast_data.set_index('Datetime',inplace=True)
solcast_data.index = solcast_data.index.tz_convert('Asia/Bangkok')
solcast_data.reset_index(inplace=True)

# Validation

In [15]:
from SolarCalculator.clearskycal import calsolar

plant = pd.read_csv('/Users/khunnoot/Desktop/year4/senior_project/solarmap_project/clear-sky-detection/plant_metadata.csv',
                   usecols = lambda column : column != 'Unnamed: 0')

In [53]:
validate_data = pd.merge(CNN_data,solcast_data, on = ['Datetime','site_name'], how = 'inner')
validate_data = validate_data.merge(plant[['site_name','lat','lng','alt']], on = 'site_name', how = 'left')
validate_data.set_index('Datetime', inplace = True)
validate_data['zna'] = calsolar.cal_dzna(validate_data, validate_data.lat.values, validate_data.lng.values)
validate_data['Iclr'] = calsolar.cal_dclearsky(validate_data, validate_data.lat.values, 
                                               validate_data.lng.values, validate_data.alt.values)
validate_data['k'] = validate_data.I/validate_data.Iclr
validate_data['date'] = validate_data.index.date

In [54]:
kmean = validate_data.groupby(by = [validate_data.index.date, 'site_name']).mean()
kmean.reset_index(inplace = True)
kmean.rename(columns = {'level_0':'date','k':'k_bar'},inplace = True)
kmean = kmean[['date','site_name','k_bar']]
validate_data.reset_index(inplace = True)
validate_data = kmean.merge(validate_data, on=['date','site_name'], how = 'right')
validate_data = validate_data[['Datetime','site_name','I','I_Cloud2Kv2','solcast_GHI','zna','k','k_bar']]
validate_data

,Datetime,site_name,I,I_Cloud2Kv2,solcast_GHI,zna,k,k_bar
0,2022-03-28 08:00:00+07:00,ISL030,123.1767,205.321342,139,64.640373,0.377315,0.799271
1,2022-03-28 08:30:00+07:00,ISL030,201.8415,208.918563,201,57.431136,0.448655,0.799271
2,2022-03-28 10:00:00+07:00,ISL030,757.8685,862.355995,694,36.153873,0.995964,0.799271
3,2022-03-28 10:30:00+07:00,ISL030,849.3851,701.815787,763,29.341230,1.015711,0.799271
4,2022-03-28 11:00:00+07:00,ISL030,977.5471,734.857742,814,22.888810,1.092901,0.799271
...,...,...,...,...,...,...,...,...
1782,2023-02-05 14:00:00+07:00,ISL048,748.0000,643.104111,367,37.201115,0.998693,0.818551
1783,2023-02-05 14:30:00+07:00,ISL048,469.0000,489.338653,368,41.958088,0.681864,0.818551
1784,2023-02-05 15:00:00+07:00,ISL048,527.0000,335.828430,338,47.400147,0.862604,0.818551
1785,2023-02-05 15:30:00+07:00,ISL048,359.0000,139.707045,378,53.317491,0.690367,0.818551


# Error matric

In [56]:
df_clr = validate_data[validate_data.k_bar >= 0.9]
df_cloud = validate_data[validate_data.k_bar < 0.9]

df_hard_time = validate_data[validate_data.zna < 50]
df_easy_time = validate_data[(validate_data.zna >= 50) & (validate_data.zna < 85)]
df_all_time = validate_data[validate_data.zna <= 85]

df_hard_clr = df_hard_time[df_hard_time.k_bar>=0.9]
df_hard_cloud = df_hard_time[df_hard_time.k_bar<0.9]

df_easy_clr =  df_easy_time[df_easy_time.k_bar>=0.9]
df_easy_cloud = df_easy_time[df_easy_time.k_bar<0.9]

df_all_clr = df_all_time[df_all_time.k_bar>=0.9]
df_all_cloud = df_all_time[df_all_time.k_bar<0.9]

In [57]:
dlist = [df_all_time, df_all_clr, df_all_cloud, df_hard_time, df_hard_clr, df_hard_cloud, df_easy_time, df_easy_clr, df_easy_cloud]

In [58]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

def single_model_eva(dz, name_model, base = 'I'):
    df = dz.copy()
    data = []
    N = len(df)
    i_mean = df[base].mean()
    df['basemodel'] = i_mean
    
    basemodel = mean_squared_error(df[base], df['basemodel'])
    
    MAE= mean_absolute_error(df[base],df[name_model])
    MSE = mean_squared_error(df[base],df[name_model])
    RMSE = np.sqrt(MSE)
    dy = df[name_model].values - df[base].values
    MBE = np.mean(dy)
    Rsq = 1- MSE/basemodel
    NMAE = MAE/i_mean*100
    NRMSE = RMSE/i_mean*100
    NMBE = MBE/i_mean*100
    SD = 100/i_mean/N*np.sqrt(N**2*MSE - np.sum(dy)**2)
    
    data.append({'MBE':MBE, 'NMBE':NMBE,
                 'MAE':MAE, 'NMAE':NMAE, 
                 'RMSE':RMSE, 'NRMSE':NRMSE, 
                 'Rsq':Rsq, 'i_mean':i_mean, 'SD':SD})
    
    df = pd.DataFrame(data)
    return df

In [70]:
name_model = 'I_Cloud2Kv2'
base = 'I'
dv1 = pd.DataFrame()
Index = ['All sky (zna 0-85 deg)',
             'Clear-sky (zna 0-85 deg)',
             'Clouded-sky (zna 0-85 deg)',
             'All sky (zna 0-50 deg)',
             'Clear-sky (zna 0-50 deg)',
             'Clouded-sky (zna 0-50 deg)',
             'All sky (zna 50-85 deg)', 
             'Clear-sky (zna 50-85 deg)', 
             'Clouded-sky (zna 50-85 deg)']
for i in range(len(dlist)):
    dv1 = pd.concat([dv1, single_model_eva(dlist[i], name_model, base = base)])
dv1.index = Index

name_model = 'solcast_GHI'
base = 'I'
dv2 = pd.DataFrame()
for i in range(len(dlist)):
    dv2 = pd.concat([dv2, single_model_eva(dlist[i], name_model, base = base)])
dv2.index = Index

In [73]:
print('Error matric of estimated irradiance (model 2)');dv1

Error matric of estimated irradiance (model 2)


,MBE,NMBE,MAE,NMAE,RMSE,NRMSE,Rsq,i_mean,SD
All sky (zna 0-85 deg),29.947783,5.664256,108.423135,20.506908,151.755556,28.702704,0.630108,528.715183,28.138256
Clear-sky (zna 0-85 deg),-8.592269,-1.306275,70.122579,10.660671,102.211583,15.539133,0.767270,657.768902,15.484130
Clouded-sky (zna 0-85 deg),44.021232,9.140823,122.409129,25.417738,166.206025,34.511979,0.541894,481.589378,33.279454
All sky (zna 0-50 deg),35.807025,5.988455,120.763162,20.196729,166.724700,27.883450,0.524758,597.934252,27.232796
Clear-sky (zna 0-50 deg),-9.832007,-1.282204,76.400025,9.963423,110.375207,14.394169,0.325863,766.805009,14.336947
Clouded-sky (zna 0-50 deg),51.236922,9.473556,135.761697,25.101938,181.868473,33.626946,0.441797,540.841491,32.264891
All sky (zna 50-85 deg),13.670732,4.063546,74.142382,22.038393,98.923786,29.404521,0.564072,336.423731,29.122387
Clear-sky (zna 50-85 deg),-5.773138,-1.408688,55.847839,13.627281,80.627394,19.673674,0.625584,409.823783,19.623176
Clouded-sky (zna 50-85 deg),22.352093,7.361092,82.310588,27.106896,106.078625,34.934293,0.470209,303.651842,34.149951


In [74]:
print('Error matric of estimated irradiance (Solcast)');dv2

Error matric of estimated irradiance (Solcast)


,MBE,NMBE,MAE,NMAE,RMSE,NRMSE,Rsq,i_mean,SD
All sky (zna 0-85 deg),42.430872,8.025280,142.643305,26.979234,198.080011,37.464408,0.369817,528.715183,36.594764
Clear-sky (zna 0-85 deg),-6.545053,-0.995038,80.108778,12.178864,105.799782,16.084643,0.750643,657.768902,16.053836
Clouded-sky (zna 0-85 deg),60.315129,12.524182,165.478679,34.360949,222.431252,46.186910,0.179527,481.589378,44.456445
All sky (zna 0-50 deg),51.568031,8.624365,159.408754,26.659913,217.845990,36.433101,0.188640,597.934252,35.397615
Clear-sky (zna 0-50 deg),-16.100189,-2.099646,84.589670,11.031445,112.890183,14.722150,0.294791,766.805009,14.571657
Clouded-sky (zna 0-50 deg),74.445678,13.764787,184.704004,34.151227,243.295563,44.984634,0.001046,540.841491,42.826953
All sky (zna 50-85 deg),17.047728,5.067338,96.068676,28.555856,128.051301,38.062506,0.269566,336.423731,37.723685
Clear-sky (zna 50-85 deg),15.183066,3.704779,69.919353,17.060833,87.564679,21.366422,0.558381,409.823783,21.042780
Clouded-sky (zna 50-85 deg),17.880269,5.888411,107.743909,35.482712,142.459604,46.915442,0.044497,303.651842,46.544445
